In [2]:
import chardet
import pandas as pd

In [3]:
with open('data/owner.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [4]:
# display full url for user_profile for instance
pd.set_option('display.max_colwidth', 1000)

In [5]:
owner = pd.read_csv("data/owner.csv",sep=';', encoding=result['encoding'])
prop = pd.read_csv("data/property.csv", sep=';', encoding=result['encoding'])

In [6]:
owner.head(4)

,Owner ID,Jour de Created At User,Jour de Last Login Date,Gender,Languages,Country,Année de Birthdate
0,0a0e4-ad50-4b3a-81fd-a29ed0000,12 mai 2018,19 février 2019,male,[],NaN,NaN
1,0a1e1-1cfe-46d0-9069-e567bc9c3,11 juin 2018,26 juin 2018,female,[],NaN,NaN
2,0a2b8-f8c2-4a2c-b6f8-d2f901abe,17 avril 2018,NaN,female,[],NaN,NaN
3,0a2bd-8842-4067-b52e-0a8922be2,26 juin 2017,17 juillet 2017,female,"[""en"", ""fr""]",NaN,NaN


In [7]:
prop.head(4)

,ID Property,ID Owner,Furnished,Max Tenant Number,Property Type,Status,Bedrooms Count,Ville,Property Surface,Rooms Count,Rent,Jour de Offline At,Jour de Online At,Jour de Created At (Properties)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0a0cd-0698-434d-a04a-339904547,392d0-46db-4c8b-80f4-c106b5ccf,FAUX,1.0,apartment,offline,NaN,Paris-Ivry-Villejuif,30,1.0,NaN,02-oct-16,03-mars-18,17-nov-16
2,0a1b8-576c-4a89-a8db-32ad0c041,48b60-51f7-4adf-8a92-1ee106c88,VRAI,2.0,apartment,offline,1.0,Toulouse,45,2.0,950,15-juil-18,24-mai-18,23-mai-18
3,0a1d1-9b9a-4ebc-b2dc-1b3609c25,503e7-6fff-4784-b0c8-ab8adda51,VRAI,1.0,house,online,1.0,NaN,10,2.0,250,NaN,03-juin-19,02-juin-19


In [8]:
print("shape of owner is :" + str(owner.shape))
print("shape of prop is :" + str(prop.shape))

shape of owner is :(15127, 7)
shape of prop is :(18686, 14)


X   Créer une colone age
   Nettoyer languages
   Calculer une colonne ancienneté
   Distribution des ages en fonction des hommes et femmes
   Scatter plot des last login
   Graph en barrer des created user
   Voir si gender correlé au last login

In [10]:
#Age design 
owner['year'] = 2019
owner['age'] = owner['year'] - owner['Année de Birthdate']

In [11]:
owner[owner["Languages"] == '{"0": "fr", "1": "en"}']

,Owner ID,Jour de Created At User,Jour de Last Login Date,Gender,Languages,Country,Année de Birthdate,year,age
1268,3cda7-ea43-4158-9a3e-74d0cf8b6,26 septembre 2017,23 octobre 2018,female,"{""0"": ""fr"", ""1"": ""en""}",NaN,NaN,2019,NaN
14841,fbc04-ee67-4d08-9651-e1e441320,11 juillet 2017,26 avril 2019,female,"{""0"": ""fr"", ""1"": ""en""}",FR,1957.0,2019,62.0


In [12]:
owner[owner["Languages"].str.startswith('{')].head(10)

,Owner ID,Jour de Created At User,Jour de Last Login Date,Gender,Languages,Country,Année de Birthdate,year,age
205,0d05d-4221-4833-87c8-6cffb9431,5 septembre 2016,27 juin 2018,female,"{""1"": ""en"", ""2"": ""fr""}",NaN,NaN,2019,NaN
733,2a0ae-3973-4758-b643-ba85bbce6,23 mai 2017,24 juillet 2018,male,"{""0"": ""en"", ""2"": ""fr"", ""3"": ""pl""}",NaN,NaN,2019,NaN
912,2ca26-8e30-4a18-b398-9a5016f40,4 octobre 2017,31 octobre 2017,female,"{""0"": ""fr"", ""1"": ""de"", ""2"": ""en_GB""}",NaN,NaN,2019,NaN
1081,2fd26-c922-45c2-8049-9401d7edf,21 décembre 2015,15 mai 2019,female,"{""1"": ""fr""}",NaN,1940.0,2019,79.0
1268,3cda7-ea43-4158-9a3e-74d0cf8b6,26 septembre 2017,23 octobre 2018,female,"{""0"": ""fr"", ""1"": ""en""}",NaN,NaN,2019,NaN
2287,6c0f3-2272-47b2-a84d-8c6c5fe6f,8 mai 2017,18 juin 2018,female,"{""1"": ""fr""}",NaN,NaN,2019,NaN
3079,8df17-b4c9-4bda-a843-2824e5003,21 juin 2016,31 mai 2019,female,"{""0"": ""fr"", ""2"": ""it""}",FR,1956.0,2019,63.0
3522,9f8e4-aaca-4420-9565-0a5d4f766,12 juillet 2017,18 juillet 2017,female,"{""1"": ""es"", ""2"": ""fr""}",NaN,NaN,2019,NaN
4255,31dbe-1cf0-4af0-bf48-ecfcca7b4,26 août 2017,13 avril 2018,female,"{""1"": ""en"", ""2"": ""es"", ""3"": ""de""}",NaN,NaN,2019,NaN
4354,35d25-7990-4f12-838d-98ed9e04a,18 juillet 2017,20 mars 2018,female,"{""1"": ""fr""}",NaN,NaN,2019,NaN


In [47]:
serie_lang = owner.Languages[owner["Languages"].str.startswith('{')]
serie_lang2 = serie_lang.str.strip("{}").str.split(":")

In [16]:
owner["Année de Birthdate"].unique()

array([  nan, 1949., 1956., 1985., 1973., 1982., 1961., 1987., 1967.,
       1970., 1947., 1965., 1966., 1963., 1952., 1944., 1972., 1978.,
       1974., 1954., 1991., 1980., 1950., 2016., 1983., 1992., 1994.,
       1939., 1960., 1971., 1958., 1953., 1959., 1948., 1977., 1968.,
       1900., 1951., 1957., 1975., 1981., 1955., 1986., 1962., 1979.,
       1989., 1990., 1976., 1964., 1993., 1945., 1997., 1940., 2005.,
       1969., 1996., 1988., 1984., 1946., 2000., 1938., 1929., 1942.,
       1943., 1999., 1941., 1998., 1933., 1995., 1926., 2030., 1937.,
       2015., 1910., 1901., 1930., 1934., 1915., 1932., 1935., 2017.,
       2971., 1936., 1928.])

In [50]:
import datetime

In [53]:
owner['today'] = datetime.date.today()
owner.head(5)

,Owner ID,Jour de Created At User,Jour de Last Login Date,Gender,Languages,Country,Année de Birthdate,year,age,today
0,0a0e4-ad50-4b3a-81fd-a29ed0000,12 mai 2018,19 février 2019,male,[],NaN,NaN,2019,NaN,2019-06-14
1,0a1e1-1cfe-46d0-9069-e567bc9c3,11 juin 2018,26 juin 2018,female,[],NaN,NaN,2019,NaN,2019-06-14
2,0a2b8-f8c2-4a2c-b6f8-d2f901abe,17 avril 2018,NaN,female,[],NaN,NaN,2019,NaN,2019-06-14
3,0a2bd-8842-4067-b52e-0a8922be2,26 juin 2017,17 juillet 2017,female,"[""en"", ""fr""]",NaN,NaN,2019,NaN,2019-06-14
4,0a3ef-f75e-463c-b52b-fa39184d5,9 mars 2018,NaN,male,[],NaN,NaN,2019,NaN,2019-06-14


In [ ]:
owner['Jour de Created At User'].date